In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv')
df

,id,escolas_postos,bairro,endereco,lat,lon,quantidade,quantidade_original,nome,subprefeitura
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL,-22.88089,-43.22533,20.0,20,CAJU,CENTRO
1,634,ESCOLA MUNICIPAL ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,-22.88957,-43.23620,121.0,121,BENFICA,CENTRO
2,483,ESCOLA MUNICIPAL CELESTINO SILVA,CENTRO,RUA DO LAVRADIO 56,-22.90929,-43.18358,220.0,220,CENTRO,CENTRO
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,-22.89763,-43.22746,190.0,190,SAO CRISTOVAO,CENTRO


In [3]:
coordinates = df[['lon', 'lat']].values
names = df['escolas_postos'].values

In [4]:
locations = df[['lon', 'lat']].values.tolist()
metrics = ['duration']

In [5]:
body = {
    "locations": locations,
    "metrics": metrics
}

In [6]:
body

{'locations': [[-43.22533, -22.88089],
  [-43.2362, -22.88957],
  [-43.18358, -22.90929],
  [-43.22746, -22.89763]],
 'metrics': ['duration']}

In [8]:
names

array(['CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL',
       'ESCOLA MUNICIPAL ALICE DO AMARAL PEIXOTO',
       'ESCOLA MUNICIPAL CELESTINO SILVA',
       'ESCOLA MUNICIPAL FLORIANO PEIXOTO'], dtype=object)

In [9]:
import requests

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-hgv', json=body, headers=headers)

print(call.status_code, call.reason)
print(call.text)

200 OK
{"durations":[[0.0,301.03,1020.37,537.25],[521.68,0.0,827.71,344.58],[703.68,706.23,0.0,637.18],[557.72,280.92,673.68,0.0]],"destinations":[{"location":[-43.22543,-22.880596],"snapped_distance":34.22},{"location":[-43.236124,-22.889655],"snapped_distance":12.23},{"location":[-43.183432,-22.909236],"snapped_distance":16.29},{"location":[-43.227804,-22.897738],"snapped_distance":37.21}],"sources":[{"location":[-43.22543,-22.880596],"snapped_distance":34.22},{"location":[-43.236124,-22.889655],"snapped_distance":12.23},{"location":[-43.183432,-22.909236],"snapped_distance":16.29},{"location":[-43.227804,-22.897738],"snapped_distance":37.21}],"metadata":{"attribution":"openrouteservice.org | OpenStreetMap contributors","service":"matrix","timestamp":1694475470778,"query":{"locations":[[-43.22533,-22.88089],[-43.2362,-22.88957],[-43.18358,-22.90929],[-43.22746,-22.89763]],"profile":"driving-hgv","responseType":"json","metrics":["duration"]},"engine":{"version":"7.1.0","build_date":"2

In [10]:
import json
data = json.loads(call.text)
durations = data['durations'] 
durations

[[0.0, 301.03, 1020.37, 537.25],
 [521.68, 0.0, 827.71, 344.58],
 [703.68, 706.23, 0.0, 637.18],
 [557.72, 280.92, 673.68, 0.0]]

In [13]:
import numpy as np
import routingpy as rp

np_durations = np.matrix(durations)
print(np_durations)

[[   0.    301.03 1020.37  537.25]
 [ 521.68    0.    827.71  344.58]
 [ 703.68  706.23    0.    637.18]
 [ 557.72  280.92  673.68    0.  ]]


In [14]:
def symmetricize(m, high_int=None):
    
    # if high_int not provided, make it equal to 10 times the max value:
    if high_int is None:
        high_int = round(10*m.max())
        
    m_bar = m.copy()
    np.fill_diagonal(m_bar, 0)
    u = np.matrix(np.ones(m.shape) * high_int)
    np.fill_diagonal(u, 0)
    m_symm_top = np.concatenate((u, np.transpose(m_bar)), axis=1)
    m_symm_bottom = np.concatenate((m_bar, u), axis=1)
    m_symm = np.concatenate((m_symm_top, m_symm_bottom), axis=0)
    
    return m_symm.astype(int) # Concorde requires integer weights

In [15]:
symmetricize(np_durations)

matrix([[    0, 10204, 10204, 10204,     0,   521,   703,   557],
        [10204,     0, 10204, 10204,   301,     0,   706,   280],
        [10204, 10204,     0, 10204,  1020,   827,     0,   673],
        [10204, 10204, 10204,     0,   537,   344,   637,     0],
        [    0,   301,  1020,   537,     0, 10204, 10204, 10204],
        [  521,     0,   827,   344, 10204,     0, 10204, 10204],
        [  703,   706,     0,   637, 10204, 10204,     0, 10204],
        [  557,   280,   673,     0, 10204, 10204, 10204,     0]])

In [16]:
from concorde.problem import Problem
from concorde.concorde import Concorde

def solve_concorde(matrix):
    problem = Problem.from_matrix(matrix)
    solver = Concorde()
    solution = solver.solve(problem)
    print(f'Optimal tour: {solution.tour}')
    return solution

In [22]:
durations_symm = symmetricize(np_durations)
solution = solve_concorde(durations_symm) # pick alternate elements: these correspond to the originals

Optimal tour: [0, 4, 1, 5, 3, 7, 2, 6]


In [23]:
tour = solution.tour[::2]

# order the original coordinates and names
coords_ordered = [coordinates[i].tolist() for i in tour]
names_ordered = [names[i] for i in tour]

In [24]:
names_ordered

['CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL',
 'ESCOLA MUNICIPAL ALICE DO AMARAL PEIXOTO',
 'ESCOLA MUNICIPAL FLORIANO PEIXOTO',
 'ESCOLA MUNICIPAL CELESTINO SILVA']

In [27]:
# add back in the first for a complete loop
coords_ordered_return = coords_ordered + [coords_ordered[0]]
coords_ordered_return

[[-43.22533, -22.88089],
 [-43.2362, -22.88957],
 [-43.22746, -22.89763],
 [-43.18358, -22.90929],
 [-43.22533, -22.88089]]

In [30]:
data = {"coordinates": coords_ordered_return}

# Convert the dictionary to a JSON-formatted string
json_data = json.dumps(data)

In [31]:
body = {"coordinates":[[8.681495,49.41461],[8.686507,49.41943],[8.687872,49.420318]]}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '',
    'Content-Type': 'application/json; charset=utf-8'
}
directions = requests.post('https://api.openrouteservice.org/v2/directions/driving-hgv', json=data, headers=headers)

print(directions.status_code, directions.reason)
print(directions.text)

200 OK
{"bbox":[-43.236124,-22.914504,-43.183059,-22.880426],"routes":[{"summary":{"distance":19147.8,"duration":2023.0000000000002},"segments":[{"distance":1849.0,"duration":301.0,"steps":[{"distance":449.2,"duration":107.8,"type":11,"instruction":"Head west on Rua Amanhecer","name":"Rua Amanhecer","way_points":[0,12]},{"distance":155.3,"duration":37.3,"type":0,"instruction":"Turn left onto Rua Interna","name":"Rua Interna","way_points":[12,18]},{"distance":397.3,"duration":44.2,"type":3,"instruction":"Turn sharp right onto Rua Péter Lund","name":"Rua Péter Lund","way_points":[18,26]},{"distance":470.1,"duration":53.7,"type":13,"instruction":"Keep right onto Viaduto Ataulfo Alves","name":"Viaduto Ataulfo Alves","way_points":[26,45]},{"distance":239.7,"duration":25.0,"type":6,"instruction":"Continue straight onto Rua Couto de Magalhães","name":"Rua Couto de Magalhães","way_points":[45,49]},{"distance":75.7,"duration":18.2,"type":1,"instruction":"Turn right onto Praça Padre Souza","name

In [37]:
directions_data = json.loads(directions.text)
directions_data

{'bbox': [-43.236124, -22.914504, -43.183059, -22.880426],
 'routes': [{'summary': {'distance': 19147.8, 'duration': 2023.0000000000002},
   'segments': [{'distance': 1849.0,
     'duration': 301.0,
     'steps': [{'distance': 449.2,
       'duration': 107.8,
       'type': 11,
       'instruction': 'Head west on Rua Amanhecer',
       'name': 'Rua Amanhecer',
       'way_points': [0, 12]},
      {'distance': 155.3,
       'duration': 37.3,
       'type': 0,
       'instruction': 'Turn left onto Rua Interna',
       'name': 'Rua Interna',
       'way_points': [12, 18]},
      {'distance': 397.3,
       'duration': 44.2,
       'type': 3,
       'instruction': 'Turn sharp right onto Rua Péter Lund',
       'name': 'Rua Péter Lund',
       'way_points': [18, 26]},
      {'distance': 470.1,
       'duration': 53.7,
       'type': 13,
       'instruction': 'Keep right onto Viaduto Ataulfo Alves',
       'name': 'Viaduto Ataulfo Alves',
       'way_points': [26, 45]},
      {'distance': 239

In [89]:
names

array(['CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL',
       'ESCOLA MUNICIPAL ALICE DO AMARAL PEIXOTO',
       'ESCOLA MUNICIPAL CELESTINO SILVA',
       'ESCOLA MUNICIPAL FLORIANO PEIXOTO'], dtype=object)

In [97]:
import folium
from folium import plugins
import polyline
import numpy as np

# Initialize the map
m = folium.Map(location=[-22.89533, -43.20957], zoom_start=13)

# Define the coordinates for the route and decode the geometry
encoded_geometry = directions_data['routes'][0]['geometry']
decoded_geometry = polyline.decode(encoded_geometry)

# Add the route polyline to the map
folium.PolyLine(
    locations=decoded_geometry,
    color='blue',
    weight=5,
    opacity=0.7,
).add_to(m)


# Create a GeoJSON object for the route
route_geojson = {
    "type": "LineString",
    "coordinates": decoded_geometry
}

# Add the route to the map
route_layer = folium.GeoJson(
    route_geojson,
    name="Route",
    style_function=lambda x: {'color': 'blue', 'weight': 5}
).add_to(m)

# Add markers for the coordinates
coordinates = np.array([[lng, lat] for lat, lng in coordinates])
for i, name in enumerate(names):
    folium.Marker(
        location=coordinates[i],
        popup=name,
        icon=folium.Icon(color='green', icon='info-sign')
    ).add_to(m)

# Add a layer control to toggle the route on/off
folium.LayerControl().add_to(m)

# Display the map
m.save('route_map.html')